# 创建用于计算和分析视域的脚本

现在，将汇总到目前为止所学的内容，并编写一个脚本来计算点的视场。随后，将分析视域属性：视域大小，可见点数以及到最近的可见点的距离。数据集是北卡罗来纳州样本数据库。

本教程将使用PyGRASS和GRASS GIS Python脚本库。

我们将根据矢量点位置，计算栅格 `elevation`的视域范围。

为了简化示例的重新运行，我们设置了此环境变量，该变量允许直接覆盖先前运行的结果，而避免覆盖检查。

In [1]:
import os
os.environ['GRASS_OVERWRITE'] = '1'

首先，利用[v.random](http://grass.osgeo.org/grass70/manuals/v.random.html)在指定区域内生成一些随机输入点（为保证再现性，在此处使用了固定种子）：


In [2]:
import grass.script as gscript

gscript.run_command('g.region', n=225200, s=222500, w=637500, e=640000, raster='elevation')#指定生成位置
gscript.run_command('v.random', output='input_points', npoints=20, seed=2, quiet=True)#生成随机点

0

## 计算一个点的视域范围

在开始遍历所有矢量点之前，我们将首先尝试单个点的计算流程。

In [3]:
from grass.pygrass.vector.geometry import Point
point = Point(638104, 223048)

现在，我们使用[r.viewshed](http://grass.osgeo.org/grass70/manuals/r.viewshed.html)从这一点计算视域，然后使用栅格代数函数[r.mapcalc](http://grass.osgeo.org/grass70/manuals/r.mapcalc.html)将所有不可见的单元格更改为空值（无数据），这是将其进一步转换为矢量数据模型（矢量化）所必需的。

In [4]:
elevation = 'elevation'
input_points = 'input_points'
viewshed_name = 'viewshed'
tmp_viewshed_name = 'tmp_viewshed'
viewshed_id = 1

gscript.run_command('r.viewshed', input=elevation, observer_elevation=3,
                    output=tmp_viewshed_name, coordinates=point.coords())
gscript.mapcalc(exp="{viewshed} = if({tmp}, {vid}, null())".format(viewshed=viewshed_name,
                                                                   tmp=tmp_viewshed_name,
                                                                   vid=viewshed_id))

视域的第一个属性是其面积。 用[r.univar](http://grass.osgeo.org/grass70/manuals/r.univar.html) 计算面积，将其作为非空像元数乘以栅格像元大小即可。 结果以报告的地图单位平方（即本例中的平方米）为单位。


In [5]:
# 统计栅格数量
cells = gscript.parse_command('r.univar', map=viewshed_name,
                              flags='g')['n']
print(cells)
print gscript.region()

18728
{u'rows': 270, u'e': 640000.0, u'projection': 99, u'zone': 0, u'cells': 67500, u'cols': 250, u'n': 225200.0, u's': 222500.0, u'w': 637500.0, u'ewres': 10.0, u'nsres': 10.0}


In [7]:
# 计算栅格数量对应的面积大小
area = float(cells) * gscript.region()['nsres'] * gscript.region()['nsres']
print(area)

1872800.0


下一个任务是查找并计算输入矢量层中包含的从当前点可见的点的数量，一种方法是通过使输入点与视域在进行空间叠加分析来导出可见点的矢量层，有关此信息，请参见[v.select](http://grass.osgeo.org/grass70/manuals/v.select.html)。但必须先使用[r.to.vect](http://grass.osgeo.org/grass70/manuals/r.to.vect.html)将视域从栅格类型转换为矢量数据模型。

In [8]:
visible_points = 'tmp_points'
gscript.run_command('r.to.vect', input=viewshed_name, output=viewshed_name,
                    type='area')#栅格转矢量
gscript.run_command('v.select', ainput=input_points, atype='point',
                    binput=viewshed_name, btype='area', 
                    operator='overlap', flags='t', output=visible_points)

0

然后可以使用[vector_info_topo](http://grass.osgeo.org/grass70/manuals/libpython/script.html?highlight=vector_info_topo#script.vector.vector_info_topo)和[v.info](http://grass.osgeo.org/grass70/manuals/v.info.html)获得视域内点的数量。

In [9]:
n_points_visible = gscript.vector_info_topo(visible_points)['points']
print n_points_visible

5


要计算的最后一个视域属性是从当前点到最近的可见点的距离，通过[v.distance](http://grass.osgeo.org/grass70/manuals/v.distance.html)进行计算，由于其输入数据要求为矢量数据，首先将现有的点图层保存下来[v.in.ascii](http://grass.osgeo.org/grass70/manuals/v.in.ascii.html)。



In [14]:
tmp_point = 'tmp_current_point' 

if float(n_points_visible) >= 1:
    gscript.write_command('v.in.ascii', input='-', stdin='%s|%s' % (point.x, point.y),
                          output=tmp_point)
    distance = gscript.read_command('v.distance', from_=tmp_point, from_type='point', flags='p',
                                    to=visible_points, to_type='point', upload='dist', dmin=1).strip()#计算当前点到上述可视范围点中的最近点的距离
    
    
    print distance
    distance = float(distance.splitlines()[1].split('|')[1])
else:
    distance = 0
print(distance)

from_cat|dist
1|953.03593896446785
953.035938964


## 多个点的视域计算

现在，将前面的代码片段放在一起，并为所有给定的输入点循环计算视域属性。

注意，由于当前点是可见点之一，因此我们要从可见点数中减去1。

In [16]:
from grass.pygrass.vector import Vector
import grass.script as gscript


elevation = 'elevation'
input_points = 'input_points'

tmp_viewshed_name = 'tmp_viewshed'
tmp_visible_points = 'tmp_points'
tmp_point = 'tmp_current_point'


with Vector(input_points, mode='r') as points:
    
    for point in points:
        viewshed_id = str(point.cat)
        viewshed_name = 'viewshed_' + viewshed_id
        gscript.run_command('r.viewshed', input=elevation, observer_elevation=3,
                            output=tmp_viewshed_name, coordinates=point.coords())
        gscript.mapcalc(exp="{viewshed} = if({tmp}, {vid}, null())".format(viewshed=viewshed_name,
                                                                           tmp=tmp_viewshed_name,
                                                                           vid=viewshed_id))

        # 可视范围面积
        cells = gscript.parse_command('r.univar', map=viewshed_name,
                                      flags='g')['n']
        area = float(cells) * gscript.region()['nsres'] * gscript.region()['nsres']


        # 可视范围内的点数量
        gscript.run_command('r.to.vect', input=viewshed_name, output=viewshed_name,
                            type='area')
        gscript.run_command('v.select', ainput=input_points, atype='point',
                            binput=viewshed_name, btype='area', 
                            operator='overlap', flags='t', output=tmp_visible_points)
        n_points_visible = gscript.vector_info_topo(tmp_visible_points)['points'] - 1
    
    
        # 到最近点的距离
        if float(n_points_visible) >= 1:
            gscript.write_command('v.in.ascii', input='-', stdin='%s|%s' % (point.x, point.y),
                                  output=tmp_point)
            distance = gscript.read_command('v.distance', from_=tmp_point, from_type='point', flags='p',
                                            to=tmp_visible_points, to_type='point', upload='dist', dmin=1).strip()

            distance = float(distance.splitlines()[1].split('|')[1])
        else:
            distance = 0
        print "%s, %d, %s, %.2f" % (viewshed_id, area, n_points_visible, distance)
    

1, 1960800, 4, 65.76
2, 678800, 0, 0.00
3, 1914400, 2, 474.09
4, 748300, 1, 1328.66
5, 639700, 1, 2026.38
6, 1184800, 1, 808.45
7, 1858600, 4, 297.82
8, 508900, 1, 164.19
9, 1037000, 2, 389.35
10, 492800, 0, 0.00
11, 114700, 0, 0.00
12, 2002100, 5, 65.76
13, 862000, 2, 808.45
14, 832300, 2, 979.21
15, 999700, 4, 389.35
16, 628800, 0, 0.00
17, 400100, 1, 1328.66
18, 1242300, 1, 2204.53
19, 1069100, 1, 164.19
20, 1902400, 6, 177.07


此处没有将结果属性打印在标准输出上，而是将它们保存到新的输出向量层的属性表中。我们通过在写模式下打开它，并通过将属性表的列作为参数传递来创建此函数。相当于将结果直接导入属性表中。

In [17]:
from grass.pygrass.vector import Vector
import grass.script as gscript


elevation = 'elevation'
input_points = 'input_points'
#
# output vector
#
output_points = 'output_points'

tmp_viewshed_name = 'tmp_viewshed'
tmp_visible_points = 'tmp_points'
tmp_point = 'tmp_current_point'

#
# define columns of the attribute table of the output vector
#
columns = [('cat', 'INTEGER'),
           ('area', 'DOUBLE PRECISION'),
           ('n_points_visible', 'INTEGER'),
           ('distance_to_closest', 'DOUBLE PRECISION')]

#
# we can open the input vector and create and open the output vector at once 
#
with Vector(input_points, mode='r') as points, \
     Vector(output_points, mode='w', tab_cols=columns) as output:

    for point in points:
        viewshed_id = str(point.cat)
        viewshed_name = 'viewshed_' + viewshed_id
        gscript.run_command('r.viewshed', input=elevation, observer_elevation=3,
                            output=tmp_viewshed_name, coordinates=point.coords())
        gscript.mapcalc(exp="{viewshed} = if({tmp}, {vid}, null())".format(viewshed=viewshed_name,
                                                                           tmp=tmp_viewshed_name,
                                                                           vid=viewshed_id))

        # viewshed size
        cells = gscript.parse_command('r.univar', map=viewshed_name,
                                      flags='g')['n']
        area = float(cells) * gscript.region()['nsres'] * gscript.region()['nsres']


        # visible points
        gscript.run_command('r.to.vect', input=viewshed_name, output=viewshed_name,
                            type='area')
        gscript.run_command('v.select', ainput=input_points, atype='point',
                            binput=viewshed_name, btype='area', 
                            operator='overlap', flags='t', output=tmp_visible_points)
        n_points_visible = gscript.vector_info_topo(tmp_visible_points)['points'] - 1
    
    
        # distance to closest visible point
        if float(n_points_visible) >= 1:
            gscript.write_command('v.in.ascii', input='-', stdin='%s|%s' % (point.x, point.y),
                                  output=tmp_point)
            distance = gscript.read_command('v.distance', from_=tmp_point, from_type='point', flags='p',
                                            to=tmp_visible_points, to_type='point', upload='dist', dmin=1).strip()

            distance = float(distance.splitlines()[1].split('|')[1])
        else:
            distance = 0

        #
        # write each point with its attributes
        # 将结果写入属性表
        output.write(point, (area, n_points_visible, distance))
        output.table.conn.commit()
        print "%s, %d, %s, %.2f" % (viewshed_id, area, n_points_visible, distance)
        
    

1, 1960800, 4, 65.76
2, 678800, 0, 0.00
3, 1914400, 2, 474.09
4, 748300, 1, 1328.66
5, 639700, 1, 2026.38
6, 1184800, 1, 808.45
7, 1858600, 4, 297.82
8, 508900, 1, 164.19
9, 1037000, 2, 389.35
10, 492800, 0, 0.00
11, 114700, 0, 0.00
12, 2002100, 5, 65.76
13, 862000, 2, 808.45
14, 832300, 2, 979.21
15, 999700, 4, 389.35
16, 628800, 0, 0.00
17, 400100, 1, 1328.66
18, 1242300, 1, 2204.53
19, 1069100, 1, 164.19
20, 1902400, 6, 177.07


最后，确保创建了新的矢量层，并正确添加了属性：

In [18]:
with Vector(output_points, mode='r') as points:
    # we can filter/sort the results
    points.table.filters.select().order_by(u'area').get_sql()
    print points.table.execute().fetchall()

[(11, 114700.0, 0, 0.0), (17, 400100.0, 1, 1328.6597708531242), (10, 492800.0, 0, 0.0), (8, 508900.0, 1, 164.19413963707365), (16, 628800.0, 0, 0.0), (5, 639700.0, 1, 2026.3822708592008), (2, 678800.0, 0, 0.0), (4, 748300.0, 1, 1328.6597700589846), (14, 832300.0, 2, 979.2095111801614), (13, 862000.0, 2, 808.4498288217176), (15, 999700.0, 4, 389.34860981442625), (9, 1037000.0, 2, 389.3486089410548), (19, 1069100.0, 1, 164.19413906795265), (6, 1184800.0, 1, 808.4498287000124), (18, 1242300.0, 1, 2204.532274362244), (7, 1858600.0, 4, 297.81998121178196), (20, 1902400.0, 6, 177.06654044228418), (3, 1914400.0, 2, 474.0895055535697), (1, 1960800.0, 4, 65.76304218300685), (12, 2002100.0, 5, 65.76304244152583)]
